
# **Movie RAG Chatbot with Hugging Face, ChromaDB, and Gemini**


This script builds a Retrieval-Augmented Generation (RAG) chatbot
from scratch for Google Colab, including a Gradio web interface.

It follows the principles from your provided notebooks:
1. Uses libraries like ChromaDB and Hugging Face embeddings.
2. Replaces the Turkish model with "`sentence-transformers/all-mpnet-base-v2`".
3. Uses the "`Pablinho/movies-dataset`" as the data source.
4. Uses `Google's Gemini API`: `gemini-2.0-flash` as the LLM.

To run this in Colab:
1. It will prompt you for your Google API Key.
2. A Gradio link will appear; click it to open the chat interface.

## **Install necessary libraries**

We install all the required packages for data loading, LangChain,vector stores, embeddings, the Google LLM, and the Gradio UI.

In [ ]:
!pip install -q \
    datasets \
    langchain \
    langchain-huggingface \
    langchain-chroma \
    langchain-text-splitters \
    langchain-google-genai \
    python-dotenv \
    gradio

print("Libraries installed successfully.")

## **Import libraries and Set Up Google API and HuggingFace Token Key**

In [ ]:
import os
import getpass
from datasets import load_dataset
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
import gradio as gr

In [ ]:
try:
    # Attempt to get API key from environment variables (useful if pre-set in Colab)
    GOOGLE_API_KEY = os.environ["GOOGLE_API_KEY"]
    print("Google API Key found in environment variables.")
except KeyError:
    # If not found in environment, prompt the user
    print("Google API Key not found in environment variables.")
    GOOGLE_API_KEY = getpass.getpass("Enter your Google API Key: ")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY # Set it for the current session

print("Google API Key set.")


In [ ]:
# --- Hugging Face Token Setup ---
HF_TOKEN = os.getenv("HF_TOKEN")
if HF_TOKEN:
    print("Hugging Face Token found in environment variables.")
else:
    print("Hugging Face Token not found in environment variables. (token=None)")

## **Load and Prepare the Dataset**

In [ ]:
documents = [] # Initialize documents list outside try block
dataset = None   # Initialize dataset outside try block
try:
    # Load the dataset from Hugging Face
    # Load the full training split
    dataset = load_dataset("Pablinho/movies-dataset", split="train")
    print(f"Loaded {len(dataset)} movies.")

    # Prepare the documents in LangChain format
    # Include Title, Overview, Genres, Rating, Release Date in page_content
    for movie in dataset:
        title = movie.get('Title', 'N/A')
        overview = movie.get('Overview', None)
        genres = movie.get('Genre', 'N/A')
        rating = movie.get('Vote_Average', 'N/A') # Keep as string/N/A for consistency in text
        release_date = movie.get('Release_Date', 'N/A')

        # Only create a document if there's an overview (primary text)
        if overview:
            # Combine desired fields for embedding
            page_content = (
                f"Title: {title}\n"
                f"Release Date: {release_date}\n"
                f"Genres: {genres}\n"
                f"Rating: {rating}\n\n"
                f"Overview: {overview}"
            )

            # Keep metadata separate as well, including Poster URL
            doc = Document(
                page_content=page_content,
                metadata={
                    "title": title,
                    "genres": genres,
                    "vote_average": rating, # Store original possibly numeric value here if needed
                    "release_date": release_date,
                    "poster_url": movie.get('Poster_Url', 'N/A') # Keep Poster URL only in metadata
                }
            )
            documents.append(doc)

    if documents:
        print(f"Created {len(documents)} LangChain Document objects from movies with overviews.")
        print("\nSample document page_content (Includes multiple fields):")
        print(documents[0].page_content[:300] + "...") # Show start of content
        print("\nSample document metadata (Includes Poster URL):")
        print(documents[0].metadata)
    else:
        print("No documents with an 'Overview' were found in the loaded sample.")

except Exception as e:
    print(f"Error loading or processing dataset: {e}")
    print("Please ensure you have an internet connection and the dataset 'Pablinho/movies-dataset' is available.")
    print("Proceeding without dataset. RAG functionality will be limited.")

## **Split Documents (Chunking)**

In [ ]:
all_splits = [] # Initialize splits list outside try block
if documents: # Only proceed if documents were loaded successfully
    try:
        # Use a text splitter to break documents into smaller chunks
        # The splitter should handle the multi-field content
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,  # Adjust if needed based on combined content length
            chunk_overlap=50
        )
        all_splits = text_splitter.split_documents(documents)

        print(f"Split {len(documents)} documents into {len(all_splits)} chunks.")
        if all_splits:
            print(f"Sample chunk (includes multiple fields): \n{all_splits[0].page_content[:200]}...")
        else:
            print("No chunks were created.")
    except Exception as e:
        print(f"Error during text splitting: {e}")
else:
    print("Skipping document splitting as no documents were loaded.")

## **Initialize Embedding Model**

In [ ]:
embeddings = None # Initialize outside try block
try:
    # Use HuggingFaceEmbeddings with all-mpnet-base-v2
    model_name = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}

    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    print(f"Embedding model '{model_name}' loaded successfully.")
except Exception as e:
    print(f"Error initializing embedding model: {e}")

## **Create Vector Store (ChromaDB)**

In [ ]:
vectorstore = None # Initialize outside try block
if all_splits and embeddings:
    try:
        # Create the vector store from the document splits
        # Embeddings now capture info from Title, Overview, Genres, Rating, Date
        vectorstore = Chroma.from_documents(
            documents=all_splits,
            embedding=embeddings,
        )
        print("Vector store created successfully (with richer content embedded).")
    except Exception as e:
        print(f"Error creating vector store: {e}")
else:
    print("Skipping vector store creation: prerequisites not met.")


## **Create the RAG Chain**

In [ ]:
# Assume necessary imports like ChatGoogleGenerativeAI, ChatPromptTemplate,
# MessagesPlaceholder, StrOutputParser, RunnablePassthrough, RunnableLambda
# and GOOGLE_API_KEY are defined earlier in your script.

llm = None      # Initialize outside try block
retriever = None  # Initialize outside try block
rag_chain = None  # Initialize outside try block

# Add a check here to see the state of vectorstore before the try block
# We assume 'vectorstore' is defined and potentially populated before this step
if 'vectorstore' in locals() or 'vectorstore' in globals():
    print(f"[Debug] Value of vectorstore before Step 7 try block: {vectorstore}")
else:
    print("[Debug] vectorstore variable not found before Step 7 try block.")
    vectorstore = None # Ensure it's defined as None if it doesn't exist

try:
    # 1. Initialize the LLM (Gemini 2.0 Flash)
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)
    print("Gemini LLM initialized.")

    # 2. Get a Retriever
    # Explicitly check if vectorstore is not None before proceeding
    if vectorstore is not None:
        try:
            retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 chunks
            print("Retriever created from vector store.")
        except Exception as e:
            print(f"Error creating retriever: {e}")
            retriever = None # Ensure retriever is None if creation fails
    else:
        print("Retriever cannot be created: vector store is unavailable (was None).")
        retriever = None # Explicitly set to None

    # Only proceed if LLM was initialized
    if llm is None:
        raise ValueError("LLM initialization failed in Step 7.") # Raise specific error

    # 3. Define a formatting function for the context (UPDATED)
    def format_docs(docs):
        if not docs:
            return "No relevant context found."
        return "\n\n".join(
            f"--- Movie ---\nTitle: {doc.metadata.get('title', 'N/A')}\n"
            f"Release Date: {doc.metadata.get('release_date', 'N/A')}\n" # Added Release Date
            f"Rating: {doc.metadata.get('vote_average', 'N/A')}\n"
            f"Genres: {doc.metadata.get('genres', 'N/A')}\n"
            f"Poster URL: {doc.metadata.get('poster_url', 'N/A')}\n" # Added Poster URL
            f"Overview: {doc.page_content}"
            for doc in docs
        )

    # 4. Create the Prompt Template for the RAG chain (UPDATED)
    qa_system_prompt = """You are a helpful and enthusiastic movie expert.
Carefully read the user's latest question ({question}) and the potentially reformulated standalone question based on chat history.
Your primary goal is to answer the question based *only* on the context provided between <context> tags.

1.  **Identify the Movie:** Try to determine the specific movie the user is asking about. Look for a movie title mentioned in the user's latest question or implied by the standalone question.
2.  **Filter Context:** If you can confidently identify the target movie, focus *only* on the context sections related to that movie's title (look for the 'Title:' field). Ignore context for other movies unless the user specifically asks for a comparison.
3.  **Answer based on Available Data:** Answer the question using *only* the filtered context for the identified movie. Remember, the context **contains Title, Release Date, Genres, Vote Average (Rating), Poster URL, and Overview**.
4.  **Handle Missing/Ambiguous Information:**
    * **If the user asks for information you DON'T have (like actors, directors, production company, etc.), even if you identified the movie:**
        * Clearly state that you don't have that *specific type* of information (e.g., "I don't have information about the actors for [Movie Title].").
        * Remind the user what information you *do* have: "I only have access to the Title, Release Date, Genres, Rating (Vote Average), Poster URL, and Overview for movies."
        * **Do NOT mention or include details from other movies in the context when responding about missing information.**

    * **If you cannot confidently determine *which* movie the user is asking about OR if the user asks for comparisons or recommendations based on rating/popularity (e.g., "Recommend some movies", "What are your most popular movies?", "Find movies with a rating above 8" or "under 5", "What kind of movie is it?"), respond as follows:**
        * **State clearly that you cannot make comparisons or filter by rating/popularity.**
        * **Explain your limitations: "I'm sorry, but I cannot compare movies or recommend them based on popularity or ratings."**
        * **Offer what you *can* do: "I can only help you find movies based on their **genre** or **overview** from the context I have."**
        * **Ask for clarification: "What kind of **genre** are you interested in (like Action, Comedy, or Sci-Fi)? Or what topics in an **overview** are you looking for?"**
        * You can optionally mention some genres present in the context if available (e.g., "Based on the current context, I see movies related to Action and Sci-Fi. Does anything like that interest you?").
    * If the context retrieved is 'No relevant context found.', inform the user you couldn't find relevant movie details.
5.  **Be Strict:** Do not make up information. Stick strictly to the provided context for the specific movie identified (if one can be identified). Do not include information about other movies from the context unless explicitly asked to compare.

<context>
{context}
</context>
"""


    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", qa_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

    # 5. Create the Prompt Template to "contextualize" questions
    contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

    # 6. Create the chain for contextualizing the question
    contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

    # 7. Create the full RAG chain logic
    def contextualized_retriever(input_dict: dict):
        question = input_dict["question"]
        chat_history = input_dict.get("chat_history", []) # Use .get for safety

        # Check if retriever exists and is functional before using it
        if retriever is None:
            print("[Debug] Retriever is None inside contextualized_retriever, returning no context.")
            return [] # Return empty list if retriever is None

        standalone_question = question
        if chat_history:
            # If there is history, reformulate the question
            standalone_question = contextualize_q_chain.invoke(
                {"question": question, "chat_history": chat_history}
            )
            print(f"[Debug: Reformulated Question: {standalone_question}]")
        else:
            print(f"[Debug: No chat history, using original question: {question}]")

        # Retrieve docs based on the standalone question
        try:
            # For general recommendation queries, retrieve diverse documents
            # For specific movie queries, retrieve documents relevant to that movie
            # (Current implementation retrieves based on standalone_question regardless)
            retrieved_docs = retriever.invoke(standalone_question)
            print(f"[Debug: Retrieved {len(retrieved_docs)} documents]")
            return retrieved_docs
        except Exception as e:
            print(f"[Error during retrieval: {e}]")
            return [] # Return empty on retrieval error


    rag_chain = (
        RunnablePassthrough.assign(
            context=RunnableLambda(contextualized_retriever) | format_docs
        )
        | qa_prompt
        | llm
        | StrOutputParser()
    )
    print("RAG chain with chat history is ready!")

except Exception as e:
    # Print the specific error that occurred during RAG chain creation
    print(f"Error creating RAG chain: {type(e).__name__}: {e}")
    print("Ensure the Gemini LLM initialized correctly and other components (like vectorstore/retriever) are available.")
    rag_chain = None # Ensure rag_chain is None if setup fails

## **Define Gradio Chat Function**

In [ ]:
print("\n--- Step 8: Defining Gradio Chat Function ---")
def chat_interface_function(message, history):
    if not message or not message.strip(): return "Please enter a question about movies."
    if not rag_chain: return "Sorry, the RAG chain is not available due to an earlier setup error."
    langchain_history = []
    for user_msg, bot_msg in history:
        if user_msg: langchain_history.append(HumanMessage(content=user_msg))
        if bot_msg: langchain_history.append(AIMessage(content=bot_msg))
    try:
        print(f"[Gradio Input: '{message}', History Length: {len(langchain_history)}]")
        response = rag_chain.invoke({"question": message.strip(), "chat_history": langchain_history})
        print(f"[Gradio Output: '{response[:100]}...']")
        return response
    except Exception as e:
        print(f"[Error in Gradio chat function]: {type(e).__name__}: {e}")
        if "API key not valid" in str(e): return "Error: Invalid Google API Key."
        elif "rate limit" in str(e).lower() or "RESOURCE_EXHAUSTED" in str(e): return "Error: API rate limit exceeded."
        elif isinstance(e, ValueError) and "LLM initialization failed" in str(e): return "Error: LLM could not be initialized."
        elif "retriever" in str(e).lower() or "vectorstore" in str(e).lower(): return "Error: Could not access movie data."
        else: return f"An unexpected error occurred: {type(e).__name__}."

## **Launch Gradio Interface**

In [ ]:
print("\n--- Step 9: Launching Gradio Interface ---")
try:
    demo = gr.ChatInterface(
        fn=chat_interface_function,
        title="🎬 Movie Chatbot (Genre/Overview Recommendations) 🍿",
        description="Ask about specific movies (Title, Overview, Rating, Genre, Date, Poster) or ask for recommendations based on genre/overview. Rating comparisons are not supported.",
        examples=[
            "What is 'Spider-Man: No Way Home' about?",
            "Tell me about 'The Batman' (2022) including its rating and poster.",
            "Recommend some action movies.",
            "Suggest movies similar to 'Inception' based on overview.",
            "Any movies about space travel?",
            "What is the rating for 'Encanto'?", # Will show rating if asked directly
            "Show me the poster for 'Scream'"
        ],
        theme="soft",
        chatbot=gr.Chatbot(height=500),
        textbox=gr.Textbox(placeholder="Ask about a movie or for genre/overview recommendations...", container=False, scale=7),
    )
    demo.launch(debug=True, share=True)
    print("Gradio interface launched.")
except Exception as e:
    print(f"Failed to launch Gradio interface: {e}")